In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum
import time

spark = SparkSession \
    .builder \
    .appName("Query 4 with 1 core/2 Gb memory") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

start_time = time.time()

# εχουμε κρατησει απο το 3ο query τους υπολογισμους για το κατα
# κεφαλην εισοδημα της καθε περιοχης
file_capita = "s3://groups-bucket-dblab-905418150721/group25/\
query4/Income per capita/"
df_income = spark.read.csv(file_capita, header=True, inferSchema=True)
# κραταμε μονο περιοχες και κατα κεφαλην εισοδημα και ταξινομουμε
# με βαση αυτο για να κρατησουμε στην συνεχεια τις τρεις πρωτες
# περιοχες και τις τρεις τελευταιες σε διαφορετικα dataframes
df_income = df_income.select("LOCATIONS", "income per capita") \
                        .sort("income per capita", ascending=False)
top3 = df_income.limit(3)
bottom3 = spark.createDataFrame(df_income.tail(3),
                                ["LOCATIONS", "income per capita"])
dis_loc = top3.union(bottom3)

# διαβαζουμε μονο το ενα συνολο δεδομενων επειδη σε αυτο εχουμε
# εγκληματα απο το 2015 και διαβαζουμε και το csv που περιεχει
# τις κωδικοποιησεις των εθνηκοτητων
file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_3 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
RE_codes.csv"
df = spark.read.csv(file_1, header=True, inferSchema=True)
df3 = spark.read.csv(file_3, header=True, inferSchema=True)
# κραταμε εγκυρες περιπτωσεις, θετικες ηλικιες, οχι null εθνικοτητες,
# και μονο συμβαντα του 2015
df = df.filter((col("Vict Age") > 0)) \
        .filter(col("Vict Descent").isNotNull()) \
        .filter(col("DATE OCC").contains("2015"))
# κραταμε τις στηλες που μας ενδιαφερουν
df = df.select("Vict Descent", "LOCATION", "Cross Street") \
        .withColumnRenamed("Cross Street", "cross_street")
# κανουμε join μονο για συμβαντα που εγιναν στις περιοχες που ζηταμε
df = dis_loc.join(df, df.LOCATION.contains(dis_loc.LOCATIONS) |
                  df.cross_street.contains(dis_loc.LOCATIONS), "inner")
# κανουμε groupBy με βαση την περιοχη και την εθνικοτητα και
# προσθετουμε τις φορες εμφανισης για καθε εθνικοτητα ξεχωριστα
df = df.groupBy("LOCATIONS", "Vict Descent") \
           .agg(count("Vict Descent").alias("#"))
# κανουμε join με τις κωδικοποιησεις ωστε να παψουμε να εχουμε
# μονο ενα γραμμα αλλα την πληρη περιγραφη της εθνικοτητας
result_df = df.join(df3, on="Vict Descent", how="inner") \
                .select("LOCATIONS", "Vict Descent Full", "#") \
                .withColumnRenamed("Vict Descent Full", "Vict Descent")
# τελος κανουμε join μια φορα για τις 3 πρωτες περιοχες και στη συνεχεια
# για τις 3 τελευταιες
result_df_fortop3 = result_df.join(top3, on="LOCATIONS", how="inner")
result_top3 = result_df_fortop3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_top3.show()

result_df_forbottom3 = result_df.join(bottom3, on="LOCATIONS", how="inner")
result_bottom3 = result_df_forbottom3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_bottom3.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3684,application_1732639283265_3630,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|               White|  9|
|               Other|  3|
|Hispanic/Latin/Me...|  2|
+--------------------+---+

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|Hispanic/Latin/Me...|533|
|               Black|305|
|               White| 48|
|               Other| 30|
|         Other Asian|  6|
|             Unknown|  3|
|              Korean|  3|
|             Chinese|  1|
|American Indian/A...|  1|
+--------------------+---+

Time taken: 58.09 seconds

In [2]:
spark = SparkSession \
    .builder \
    .appName("Query 4 with 2 core/4 Gb memory") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

start_time = time.time()

# εχουμε κρατησει απο το 3ο query τους υπολογισμους για το κατα
# κεφαλην εισοδημα της καθε περιοχης
file_capita = "s3://groups-bucket-dblab-905418150721/group25/\
query4/Income per capita/"
df_income = spark.read.csv(file_capita, header=True, inferSchema=True)
# κραταμε μονο περιοχες και κατα κεφαλην εισοδημα και ταξινομουμε
# με βαση αυτο για να κρατησουμε στην συνεχεια τις τρεις πρωτες
# περιοχες και τις τρεις τελευταιες σε διαφορετικα dataframes
df_income = df_income.select("LOCATIONS", "income per capita") \
                        .sort("income per capita", ascending=False)
top3 = df_income.limit(3)
bottom3 = spark.createDataFrame(df_income.tail(3),
                                ["LOCATIONS", "income per capita"])
dis_loc = top3.union(bottom3)
# διαβαζουμε μονο το ενα συνολο δεδομενων επειδη σε αυτο εχουμε
# εγκληματα απο το 2015 και διαβαζουμε και το csv που περιεχει
# τις κωδικοποιησεις των εθνηκοτητων
file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_3 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
RE_codes.csv"
df = spark.read.csv(file_1, header=True, inferSchema=True)
df3 = spark.read.csv(file_3, header=True, inferSchema=True)
# κραταμε εγκυρες περιπτωσεις, θετικες ηλικιες, οχι null εθνικοτητες,
# και μονο συμβαντα του 2015
df = df.filter((col("Vict Age") > 0)) \
        .filter(col("Vict Descent").isNotNull()) \
        .filter(col("DATE OCC").contains("2015"))
# κραταμε τις στηλες που μας ενδιαφερουν
df = df.select("Vict Descent", "LOCATION", "Cross Street") \
        .withColumnRenamed("Cross Street", "cross_street")
# κανουμε join μονο για συμβαντα που εγιναν στις περιοχες που ζηταμε
df = dis_loc.join(df, df.LOCATION.contains(dis_loc.LOCATIONS) |
                  df.cross_street.contains(dis_loc.LOCATIONS), "inner")
# κανουμε groupBy με βαση την περιοχη και την εθνικοτητα και
# προσθετουμε τις φορες εμφανισης για καθε εθνικοτητα ξεχωριστα
df = df.groupBy("LOCATIONS", "Vict Descent") \
           .agg(count("Vict Descent").alias("#"))
# κανουμε join με τις κωδικοποιησεις ωστε να παψουμε να εχουμε
# μονο ενα γραμμα αλλα την πληρη περιγραφη της εθνικοτητας
result_df = df.join(df3, on="Vict Descent", how="inner") \
                .select("LOCATIONS", "Vict Descent Full", "#") \
                .withColumnRenamed("Vict Descent Full", "Vict Descent")
# τελος κανουμε join μια φορα για τις 3 πρωτες περιοχες και στη συνεχεια
# για τις 3 τελευταιες
result_df_fortop3 = result_df.join(top3, on="LOCATIONS", how="inner")
result_top3 = result_df_fortop3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_top3.show()

result_df_forbottom3 = result_df.join(bottom3, on="LOCATIONS", how="inner")
result_bottom3 = result_df_forbottom3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_bottom3.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|               White|  9|
|               Other|  3|
|Hispanic/Latin/Me...|  2|
+--------------------+---+

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|Hispanic/Latin/Me...|533|
|               Black|305|
|               White| 48|
|               Other| 30|
|         Other Asian|  6|
|             Unknown|  3|
|              Korean|  3|
|             Chinese|  1|
|American Indian/A...|  1|
+--------------------+---+

Time taken: 26.66 seconds

In [3]:
spark = SparkSession \
    .builder \
    .appName("Query 4 with 4 core/8 Gb memory") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

start_time = time.time()

# εχουμε κρατησει απο το 3ο query τους υπολογισμους για το κατα κεφαλην
# εισοδημα της καθε περιοχης
file_capita = "s3://groups-bucket-dblab-905418150721/group25/query4/\
Income per capita/"
df_income = spark.read.csv(file_capita, header=True, inferSchema=True)
# κραταμε μονο περιοχες και κατα κεφαλην εισοδημα και ταξινομουμε
# με βαση αυτο για να κρατησουμε στην συνεχεια τις τρεις πρωτες
# περιοχες και τις τρεις τελευταιες σε διαφορετικα dataframes
df_income = df_income.select("LOCATIONS", "income per capita") \
                        .sort("income per capita", ascending=False)
top3 = df_income.limit(3)
bottom3 = spark.createDataFrame(df_income.tail(3),
                                ["LOCATIONS", "income per capita"])
dis_loc = top3.union(bottom3)
# διαβαζουμε μονο το ενα συνολο δεδομενων επειδη σε αυτο εχουμε
# εγκληματα απο το 2015 και διαβαζουμε και το csv που περιεχει
# τις κωδικοποιησεις των εθνηκοτητων
file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_3 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
RE_codes.csv"
df = spark.read.csv(file_1, header=True, inferSchema=True)
df3 = spark.read.csv(file_3, header=True, inferSchema=True)
# κραταμε εγκυρες περιπτωσεις, θετικες ηλικιες, οχι null εθνικοτητες,
# και μονο συμβαντα του 2015
df = df.filter((col("Vict Age") > 0)) \
        .filter(col("Vict Descent").isNotNull()) \
        .filter(col("DATE OCC").contains("2015"))
# κραταμε τις στηλες που μας ενδιαφερουν
df = df.select("Vict Descent", "LOCATION", "Cross Street") \
        .withColumnRenamed("Cross Street", "cross_street")
# κανουμε join μονο για συμβαντα που εγιναν στις περιοχες που ζηταμε
df = dis_loc.join(df, df.LOCATION.contains(dis_loc.LOCATIONS) |
                  df.cross_street.contains(dis_loc.LOCATIONS), "inner")
# κανουμε groupBy με βαση την περιοχη και την εθνικοτητα και
# προσθετουμε τις φορες εμφανισης για καθε εθνικοτητα ξεχωριστα
df = df.groupBy("LOCATIONS", "Vict Descent") \
           .agg(count("Vict Descent").alias("#"))
# κανουμε join με τις κωδικοποιησεις ωστε να παψουμε να εχουμε
# μονο ενα γραμμα αλλα την πληρη περιγραφη της εθνικοτητας
result_df = df.join(df3, on="Vict Descent", how="inner") \
                .select("LOCATIONS", "Vict Descent Full", "#") \
                .withColumnRenamed("Vict Descent Full", "Vict Descent")
# τελος κανουμε join μια φορα για τις 3 πρωτες περιοχες και στη συνεχεια
# για τις 3 τελευταιες
result_df_fortop3 = result_df.join(top3, on="LOCATIONS", how="inner")
result_top3 = result_df_fortop3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_top3.show()

result_df_forbottom3 = result_df.join(bottom3, on="LOCATIONS", how="inner")
result_bottom3 = result_df_forbottom3.groupBy("Vict Descent") \
                                .agg(sum("#").alias("#")) \
                                .sort("#", ascending=False) \
                                .withColumnRenamed("Vict Descent",
                                                   "Victim Descent")
result_bottom3.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|               White|  9|
|               Other|  3|
|Hispanic/Latin/Me...|  2|
+--------------------+---+

+--------------------+---+
|      Victim Descent|  #|
+--------------------+---+
|Hispanic/Latin/Me...|533|
|               Black|305|
|               White| 48|
|               Other| 30|
|         Other Asian|  6|
|             Unknown|  3|
|              Korean|  3|
|             Chinese|  1|
|American Indian/A...|  1|
+--------------------+---+

Time taken: 26.02 seconds